# Title

# Introduction:
- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question
3 continuous variables, 3 binary variables, 14 categorical variables; 
This dataset includes 173 species of mushrooms with caps from various families and one
	entry for each species.

# Preliminary exploratory data analysis:
- Demonstrate that the dataset can be read from the web into R 

In [38]:
library(tidyverse)
library(tidymodels)
mushrooms <- read_delim("data\ folder/secondary_data.csv", delim=";")
head(mushrooms)

Rows: 61069 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (16): class, cap-shape, cap-surface, cap-color, gill-attachment, gill-sp...
dbl  (3): cap-diameter, stem-height, stem-width
lgl  (2): does-bruise-or-bleed, has-ring

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,⋯,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
<chr>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
p,15.26,x,g,o,FALSE,e,NA,w,16.95,⋯,s,y,w,u,w,TRUE,g,NA,d,w
p,16.60,x,g,o,FALSE,e,NA,w,17.99,⋯,s,y,w,u,w,TRUE,g,NA,d,u
p,14.07,x,g,o,FALSE,e,NA,w,17.80,⋯,s,y,w,u,w,TRUE,g,NA,d,w
p,14.17,f,h,e,FALSE,e,NA,w,15.77,⋯,s,y,w,u,w,TRUE,p,NA,d,w
p,14.64,x,h,o,FALSE,e,NA,w,16.53,⋯,s,y,w,u,w,TRUE,p,NA,d,w
p,15.34,x,g,o,FALSE,e,NA,w,17.84,⋯,s,y,w,u,w,TRUE,p,NA,d,u


- Clean and wrangle your data into a tidy format

In [51]:
mushroom_wrangle <- mushrooms |>
    select(class, "cap-diameter", "stem-height", "stem-width") |>
    mutate(class = as_factor(class)) 
colnames(mushroom_wrangle) <- c("class", "diameter", "height", "width")

mushroom_split <- mushroom_wrangle |>
    initial_split(prop=0.75, strata=class)

mushroom_training <- training(mushroom_split)
mushroom_training

[1] 45801

[1] 45801

- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 

In [53]:
summarize_mushroom <- mushroom_training |>
    group_by(class) |>
    summarize(n = n())
summarize_mushroom

means_mushroom <- mushroom_training |>
    select(-class) |>
    map_df(mean)
    means_mushroom

missing_data <- mushroom_training |>
    filter(is.na(diameter)) |>
    nrow()
missing_data

class,n
<fct>,<int>
p,25416
e,20385


diameter,height,width
<dbl>,<dbl>,<dbl>
6.71671,6.573344,12.12319


[1] 0

- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

# Methods:
- Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
- Describe at least one way that you will visualize the results

# Expected outcomes and significance:
- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?